# Easily use Snowflake in Existing ML Tool & Pipelines

Customers who use Snowflake with AzureML or AWS Sagemaker will have models trained and registered in either cloud ML tool but may prefer to deploy those models to Snowflake for batch inference.
The Snowflake python libraries can run anywhere and can be easily added to existing 3rd party tools, see examples below.

Advantages to deploying models in Snowflake include: 

- Bringing the model to the data. If all (or most) of your inference data is in Snowflake then having the model with the data makes for a more efficient and secure experience. 
- Data doesn't leave the Snowflake platform.
- Security and Flexibility. With the model deployed in the SnowparkML Registry admins can leverage Snowflake's RBAC to easily control access and organizations have the flexibility to allow privileged users the ability to generate predictions with their Snowpark model.
- Ease of Use & Scalability

# Register a model in Snowflake from SageMaker

In [ ]:
# import packages
import boto3
from sagemaker import get_execution_role
import sagemaker
from joblib import load

# create clients for S3 and sagemaker
s3 = boto3.client("s3")
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()


# using model arn from model registry describe model details
sm_client = boto3.Session().client('sagemaker')
sm_client.describe_model_package(ModelPackageName='<model arn>')

# download model from S3 using information from above
s3_client = boto3.client('s3')
s3_client.download_file('<bucket from ModelDataUrl>', 'model file path from ModelDataURL', '<new model name in local sagemaker env>.tar.gz')

# unzip and load model
!tar -xf '<new model name in local sagemaker env>.tar.gz' -C .
sm_model = load("model.joblib")
sm_model

# connect to Snowflake with Snowpark
import pandas as pd
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
from snowflake.ml.registry import registry

connection_parameters = {
    "account": "<locator>", # e.g. xy12345.us-east-2.aws 
    "user": "<username>", 
    "password": "<password>",
    "role": "<role>",
    "warehouse": "<virtual warehouse>",
    "database": "<database>",
    "schema": "<schema>"
    }
session = Session.builder.configs(connection_parameters).create()

# connect to Snowpark registry and log model
reg = registry.Registry(session=session)
reg.log_model(sm_model, model_name='<name of model in Snowflake>', version_name='v1', sample_input_data=<sample dataframe>)

# verify model deployment and view functions associated with the model
mv = reg.get_model('<name of model in Snowflake>').version('v1')
mv.show_functions()


# Register a model in Snowflake from AzureML

In [ ]:
from azureml.core import Workspace, Dataset
import numpy as np

# Connect to the Workspace
ws = Workspace.from_config()

# Access model using model name
from azureml.core import Model
model_path = Model.get_model_path(model_name = '<model name>', version = 1, _workspace= ws)
model_path

# Load model
from joblib import load
aml_model = load(model_path + "/model.pkl")
aml_model

# Look at model environment
with open(model_path +  '/./python_env.yaml') as f:
    print(f.read())

# Look at model requirements
with open(model_path +  '/./requirements.txt') as f:
    print(f.read())

# connect to Snowflake with Snowpark
import pandas as pd
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
from snowflake.ml.registry import registry

connection_parameters = {
    "account": "<locator>", # e.g. xy12345.us-east-2.aws 
    "user": "<username>", 
    "password": "<password>",
    "role": "<role>",
    "warehouse": "<virtual warehouse>",
    "database": "<database>",
    "schema": "<schema>"
    }
session = Session.builder.configs(connection_parameters).create()

# connect to Snowpark registry and log model
reg = registry.Registry(session=session)
reg.log_model(aml_model, model_name='<name of model in Snowflake>', version_name='v1', sample_input_data=<sample dataframe>)

# verify model deployment and view functions associated with the model
mv = reg.get_model('<name of model in Snowflake>').version('v1')
mv.show_functions()

# Build and Register a Model all here in Snowflake

https://github.com/Snowflake-Labs/sf-samples/tree/main/samples/ml?_fsi=wWDeqSCS&_fsi=wWDeqSCS&_fsi=wWDeqSCS&_fsi=wWDeqSCS